# MPA 5830 - Module 02 (Fall 2021)
In this module our goal is to understand how some very powerful packages work their magic. In particular, although they are of somewhat recent vintage, `{dplyr}`, `{tidyr}` and `{lubridate}` have quickly gained a fan following because they allow you to clean and organize your data and then calculate quantities of interest with surprising ease. In this module we will start to see some of these packages' core functionalities and wrap up this particular leg of our learning journey in __`Module 03`__. 

## `{dplyr}` 
There is a common quote that is tossed about a good bit in the hallways of data science, that, and I am paraphrasing here, a data scientists spends about 80% of their time gathering, cleaning, and organizing their data and spends only about 20% of their time on the analysis per se. This may or may not be true, especially in the initial stages of a new project but yes, we do spend an awfully large amount of our time getting the data ready for visualizations and other analyses. You do this work long enough and you come to realize that anything you could do to speed up the cleaning phase would be time and money saved. And yet, data cleaning skills are in short supply. On the plus side of the ledger, packages like `{dplyr}` and `{data.table}` have simplified what were once nightmarish tasks. 

`Nightmare` is not a word to be tossed around lightly and so let us setup a seemingly large data-set with 100+ columns, and tons of information hidden in it. Once we setup this scenario, spell out a few questions we would like to answer, we might better appreciate how `{dplyr}` comes to our aid. In particular, we might come to understand how `{dplyr}` uses seven core verbs:  

| **What you want to do ...** | **`{dplyr}` function** |
| :-----  | :----- |
| you need to select columns to work with? | `select()`|
| you need to use a subset of the data based on some criterion? | `filter()`|
| you need to arrange the data in ascending/descending order of variable(s)? | `arrange()`|
| you want the results of your calculations to be a standalone data frame? | `summarise()`|
| you want to add your calculated value(s) to the existing data frame? | `mutate()`|
| you want to add your calculated value(s) to the existing data frame but also  drop all  variables/columns not  being used in the calculation? | `transmute()`|
| you need to calculate averages, frequencies, etc by groups? | `group_by()`|

Here is the dataset -- all flights originating and departing from Columbus (Ohio) January through September of 2017. 

Let  us load the data, and the `{tidyverse}` and `{here}` packages. 

In [ ]:
library(tidyverse) 

load("data/cmhflights_01092017.RData")

In [ ]:
names(cmhflights) # will show you the column names

The output is rather onerous with 110 columns displayed, the last being `X110`, an empty column that can be dropped. 

In [ ]:
cmhflights$X110 <- NULL #This command will delete the column named X110 

Did it work? Let us check.

In [ ]:
names(cmhflights)

Now we are down to 109 columns and ready to get to work. 

## select() 
As in the present case of 100+ columns, often our data frame will have more columns than we plan to work with. In such instances it is a good idea to drop all unwanted columns; this will make the data-set more manageable and tax our computing resources less. For example, say I only want the first five columns (which happen to be `Year`, `Quarter`, `Month`, `DayofMonth`, and `DayOfWeek`). I could use `select` to create a data frame with only these columns:

In [ ]:
cmhflights %>% 
  select(Year:DayOfWeek) -> my.df

The `:` is the bridge between __consecutive columns__ starting with `Year` and stopping with `DayofWeek`.

In addition, note that the subset of columns selected and all rows of data are being written to a new data frame called `my.df`

Quick check to see if we have only the columns we wanted. 

In [ ]:
names(my.df)

Wonderful! It worked.

Now, the same result could have been obtained by taking the longer route of __listing each column by name__, as in the following: 

In [ ]:
cmhflights %>%
  select(Year, Quarter, Month, DayofMonth, DayOfWeek) -> my.df 

names(my.df)

What if the columns were not sequentially located? In that case we would need to list each column we want. Say I want `Year`, `FlightDate`, `UniqueCarrier`, and `TailNum`. 

In [ ]:
cmhflights %>% 
  select(Year, FlightDate:UniqueCarrier, TailNum) -> my.df 

names(my.df)

Could we have used __column numbers instead of column names__? Absolutely. 

In [ ]:
cmhflights %>% 
  select(c(1, 3, 5, 7)) -> my.df 

names(my.df)

You can also use __consecutive column numbers__, for examples, columns 1 through 5 as follows: 

In [ ]:
cmhflights %>% 
  select(c(1:5)) -> my.df 

names(my.df)

You can also use column numbers to select a mix of columns, some that may be sequential and others that may be not sequential.

In [ ]:
cmhflights %>% 
  select(c(1, 6:9, 12)) -> my.df

names(my.df)

### select() in other ways
We can also select columns in other ways, by specifying that  the column name `contain` some element. The code below shows you how this is done if I am looking for column names with the phrase "Carrier", then with "De", and then with "Num"  

In [ ]:
cmhflights %>% 
  select(contains("Carrier")) -> my.df

names(my.df)

You can also specify that the columns to be selected __start with__ some alphanumeric string, for example, 

In [ ]:
cmhflights %>% 
  select(starts_with("De")) -> my.df

names(my.df)

The other option would be to choose columns that __end with__ a particular alphanumeruc string, for example, 

In [ ]:
cmhflights %>% 
  select(ends_with("Num")) -> my.df

names(my.df)

There are two other options -- `matches()` and `num_range()`. Let us look at each in turn. 

In [ ]:
cmhflights %>%
    select(matches("el")) -> my.df

names(my.df)

Here is another dataset where some of the column names contain a number.

In [ ]:
head(billboard)

In [ ]:
billboard %>%
    select(num_range("wk", 1:5)) -> my.df

names(my.df)

------------------

## filter()
Do you really want all the rows in the data-set or do you want to see only very specific rows that meet some criteria? Say we only want to look at certain months, or only flights on Saturdays and Sundays, or flights in a given month. For example, say we only want flights in January, i.e., `Month == 1`. 

In [ ]:
cmhflights %>% 
  filter(Month == 1) -> my.df

table(my.df$Month) # Show me a frequency table for Month in my.df

What about only American Airline flights in January? Note that the UniqueCarrier code for this airline is AA.

In [ ]:
cmhflights %>% 
  filter(Month == 1, UniqueCarrier == "AA") -> my.df 

table(my.df$Month, my.df$UniqueCarrier) # a simple frequency table

Note that `,` inside `filter` means `&`

What about United Airlines flights in January to CMH (the airport code for Columbus, OH), regardless of where the flight originated? 

In [ ]:
cmhflights %>% 
  filter(Month == 1, UniqueCarrier == "UA", Dest == "CMH") -> my.df

head(my.df)

What if I wanted a more complicated filter, say, flights in January or February, either to CMH or to ORD (the airport code for O'Hare in Chicago)?

In [ ]:
cmhflights %>% 
  filter(
    Month %in% c(1, 2), UniqueCarrier == "UA", Dest %in% c("CMH", "ORD")
    ) -> my.df 

table(my.df$Month) # frequency table of Month

In [ ]:
table(my.df$UniqueCarrier) # frequency table of UniqueCarrier

In [ ]:
table(my.df$Dest) # frequency table of Dest

In [ ]:
cmhflights %>% 
  filter(
    !Month %in% c(1, 2)
    ) -> my.df 

table(my.df$Month)

In [ ]:
cmhflights %>%
filter(
    Month != 1, Month != 2
    ) -> my.df

table(my.df$Month)

Beautiful, just beautiful. 

At this point it may not be readily apparent to you but using `%in% c(...)` makes applying complex filters easier than if you go some other route. 

Before we move on, note the operators  that work with `filter()` 

| **Operator** | **Meaning**  | **Operator** | **Meaning** |
| :---- | :---- | :---- |  :---- |
| $<$    | less than | $>$    | greater than |
| $==$   | equal to  | $\leq$ | less than or equal to |
| $\geq$ | greater than or equal to | `!=` | not equal to |
| `%in%`   | is a member of |  `is.na` | is NA |
| `!is.na` | is not NA  | `&,!,etc` | Boolean operators |

----------------

## arrange()

Now let us say I wanted to arrange the resulting data frame in `ascending order` of departure delays. How might I do that? Via `arrange()` 

Before we see this new command in action I am going to whittle the data-frame to only a few columns, and only flights to CMH or ORD. That will make it easier to see the result of executed commands.

In [ ]:
cmhflights %>% 
    select('Year', 'Month', 'DayofMonth', 'FlightDate', 'Carrier', 'TailNum', 'FlightNum', 
           'Origin', 'OriginCityName', 'Dest', 'DestCityName', 'CRSDepTime', 'DepTime', 
           'DepDelay', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes') %>%
    filter(Dest %in% c("CMH", "ORD")) -> my.df

my.df %>% 
  arrange(DepDelayMinutes) -> my.df2

my.df2

And now in `descending order` of delays by adding the minus symbol `-` to the  column name. 

In [ ]:
my.df %>% 
  arrange(-DepDelayMinutes) -> my.df2

my.df2

We could tweak this further, perhaps saying sort by departure delays to CMH, and then to ORD. 

In [ ]:
my.df %>% 
  arrange(Dest, -DepDelayMinutes) -> my.df2

my.df2

So far, we have seen each function in isolation. Now we streamline things a bit so that we only end up with the columns and rows we want to work with, arranged as we want the resulting data-set to be. 

In [ ]:
cmhflights %>% 
  select(Month, UniqueCarrier, Dest, DepDelayMinutes) %>% 
  filter(
    Month %in% c(1, 2), UniqueCarrier == "UA", Dest %in% c("CMH", "ORD")
    ) %>% 
  arrange(Month, Dest, -DepDelayMinutes) -> my.df3

my.df3

Here, the end result is a data frame arranged by Month, then within Month by Destination, and then finally by descending order of flight delays. This is the beauty of `dplyr`, allowing us to chain together various functions to get what we want. How is this helpful? Well, now you have a data frame that you can analyze. What do I want to calculate? Well, let us say we want to create a frequency table, something that we would have done with Excel via a Pivot Table. 

------------

## summarise() or summarize()
What if we need to calculate frequencies? For example, how many flights per month are there? What if we want the __mean__ DepDelay or __median__ ArrDelay? These can be easily calculated as shown below. 

Let us start with frequencies.

In [ ]:
cmhflights %>%
  count(Month) # Most flights were in July (n = 4295)

What about by days of the week AND by month? 

In [ ]:
cmhflights %>%
  count(Month, DayOfWeek) # Output is sorted by Month and then DayOfWeek

I want to know the average departure delay and the average arrival delay for all flights, with the averages calculated in two ways -- as the `mean` and as the `median`. Maybe I also want the variance and the standard deviation of both delays. 

In [ ]:
cmhflights %>%
  summarise(
      mean_arr_delay = mean(ArrDelay, na.rm = TRUE),
      mean_dep_delay = mean(DepDelay, na.rm = TRUE),
      median_arr_delay = median(ArrDelay, na.rm = TRUE),
      median_dep_delay = median(DepDelay, na.rm = TRUE),
      variance_arr_delay = var(ArrDelay, na.rm = TRUE),
      variance_dep_delay = var(DepDelay, na.rm = TRUE),  
      sd_arr_delay = sd(ArrDelay, na.rm = TRUE),
      sd_dep_delay = sd(DepDelay, na.rm = TRUE)
  )

Here, the `na.rm = TRUE` command is useful because R will not allow you to calculate any mean, median, variance, etc. if the clumn includes some rows with missing data. 

You can see this below, where I have a small data-set called `df` with 4 values of x, but one of the four is missing and recorded as `NA`. See what happens when I try to calculate the mean with/without `na.rm = TRUE`.

In [ ]:
df = data.frame(x = c(2, 4, 9, NA))

df

In [ ]:
df %>%
  summarise(mean.x = mean(x)) # You get no meaningful values 

In [ ]:
df %>%
  summarise(mean.x = mean(x, na.rm = TRUE)) # Now you do get something meaningful

What if I want to count total DepDelay by airline? 

In [ ]:
cmhflights %>%
    group_by(Carrier) %>%
    summarize(sum(DepDelay, na.rm = TRUE))

--------------

## group_by()
These summaries are fine if you want to calculate aggregate quantities of interest (i.e., means, medians, frequencies, variances, etc. for all rows of data) but what if you wanted to calculate the __number of flights per month, by airline? Average delays by airline?__ 

Now things get interesting because `group_by()` will open up this new world for us! The first thing I will calculate is the number of flights by airline per month. 

In [ ]:
cmhflights %>%
  group_by(Month, Carrier) %>%
  tally()

You can get the same result if you use `summarize()`

In [ ]:
cmhflights %>%
  group_by(Month, Carrier) %>%
  summarize(
    frequency = n()
    )

Top recap, two ways to do this, first with `tally()` and the second with `summarize(frequency = n())` ... both yielding the same result. Remember `tally()` because it is shorter code. 

Now I want a table that gives us the number of flights per month per airline per destination. 

In [ ]:
cmhflights %>%
  group_by(Month, Carrier, Dest) %>%
  tally()

We could keep enriching the grouping structure. For example, let us add the day of the week to the mix ... 

In [ ]:
cmhflights %>%
  group_by(Month, Carrier, Dest, DayOfWeek) %>%
  tally()

Now say I am really curious about mean departure delays for the preceding grouping structure. That is, what does mean departure delay look like for flights by day of the week, by month, by carrier, and by destination?

In [ ]:
cmhflights %>%
  group_by(Month, Carrier, Dest, DayOfWeek) %>%
  summarise(mean_dep_delay = mean(DepDelay, na.rm = TRUE))

But this is a complicated summary table. What if all I really want to know is what airline has the __highest mean departure delays__, regardless of month or destination or day of the week? This could be done by using `arrange()` to display the result in descending order of _mean_dep_delay_

In [ ]:
cmhflights %>%
  group_by(Carrier) %>%
  summarise(mean_dep_delay = mean(DepDelay, na.rm = TRUE)) %>%
  arrange(-mean_dep_delay) # ordered in descending order of delays

EV is Express Jet; F9 is Frontier Airlines; WN is Southwest Airlines; OO is SkyWest Airlines; AA is American Airlines; DL is Delta Airlines; UA is United Airlines. So clearly United Airlines had the lowest average departure delays. Would this still be true if we repeated the calculation by Month? 

In [ ]:
cmhflights %>%
  group_by(Carrier, Month) %>%
  summarise(mean_dep_delay = mean(DepDelay, na.rm = TRUE)) %>%
  arrange(mean_dep_delay) # ordered in descending order of delays

All righty then! Looks like three of the lowest mean departure delays were for SkyWest. Do not let the negative numbers throw you for a loop; a negative value implies the flight left earlier than scheduled. 

So far so good. But now I am curious about what percent of flights operated by AA, DL, UA, and WN were delayed. How could I calculate this? 

(1) I need to use `filter()` to restrict the data-set to just these four airlines.  
(2) Then I need to generate a new column that identifies whether a flight was delayed or not (`late`).  
(3) Now we can calculate the total number of flights (`nflights`) and the total number of flights that were delayed (`nlate`).  
(4) If I then calculate $\left( \dfrac{nlate}{nflights} \right)\times 100$ we will end up with the percent of flights that were delayed.  

In [ ]:
cmhflights %>%
  select(c(Carrier, DepDelay)) %>%
  filter(
      Carrier %in% c("AA", "DL", "UA", "WN")
  ) %>%
  mutate(
      late = case_when(
          DepDelay > 0 ~ "Yes",
          DepDelay <= 0 ~ "No"
      )
  ) %>% 
  group_by(Carrier) %>%
  mutate(
      nflights = n()
  ) %>%
  group_by(Carrier, late) %>%
  mutate(
    nlate = n(),
    pct_late = (nlate / nflights) * 100
  ) -> df1

df1

There is a whole lot going on here so let us break it down. 

`filter(Carrier %in% c("AA", "DL", "UA", "WN"))` is keeping specified airlines' data while dropping the rest

`mutate(late = case_when(...)` is creating a new column called `late` and storing a value of "Yes" if `DepDelay > 0` (i.e., the flight was delayed by 1 or more minutes) and "No" if `DepDelay <= 0` (i.e., if the flight departed on time or earlier than the scheduled departure time) 

`group_by(Carrier)` is grouping by Carrier and then counting with `mutate(nflights = n())` how many flights there were per Carrier and storing this sum in a new column called `nflights`

`group_by(Carrier, late)` then regroups the data, this time by Carrier and if the flight was late or not


`mutate(nlate = n(), pct_late = (nlate / nflights) * 100)` is then creating two new columns, `nlate` -- the number of flights per late values of "Yes" and "No", respectively, and then `pct_late` -- the percent of flights per carrier that were late.  

Now, we only want the flights that were late so let us apply `select()` to keep just a few columns and then we use `filter()` to keep only rows corresponding to `late = "Yes"`. 

This will still leave us with duplicate rows but we can drop these duplicate rows via a new command, `distinct()`  -- which if left empty inside the parentheses `()` looks for all unique rows of data (each row ends up with a unique combination of all columns' values). If you want unique values of specific columns then those column names can be inserted inside the parentheses `()`

In [ ]:
df1 %>%
  filter(late == "Yes") %>%
  ungroup() %>%
  select(Carrier, pct_late) %>%
  distinct() %>%
  arrange(pct_late)

So! 24% of UA flights were late, the lowest in this group. 

What if we wanted to do this for all airlines, and we want the calculations to be done by Month as well?

In [ ]:
cmhflights %>%
  select(c(Carrier, Month, DepDelay)) %>%
#  filter(Carrier %in% c("AA", "DL", "UA", "WN")) %>%
  mutate(late = case_when(
    DepDelay > 0 ~ "Yes",
    DepDelay <= 0 ~ "No"
      )
    ) %>% 
  group_by(Carrier, Month) %>%
  mutate(nflights = n()) %>%
  group_by(Carrier, Month, late) %>%
  mutate(
    nlate = n(),
    pct_late = (nlate / nflights) * 100) %>%
  filter(late == "Yes") %>%
  ungroup() %>%  
  select(Carrier, Month, pct_late) %>%
  distinct() %>%
  arrange(pct_late)

Before we move on, I want to point out something about `case_when()`. Specifically, we used it to create a new column called `late` from numeric values found in `DepDelay`. But what if we wanted to create a new column from a column that had categorical variables in it, like `Dest` or `Carrier`? Easy.

In [ ]:
cmhflights %>%
    mutate(
      carrier_name = case_when(
          Carrier == "AA" ~ "American Airlines",
          Carrier == "DL" ~ "Delta Airlines",
          Carrier == "UA" ~ "United Airlines",
          Carrier == "EV" ~ "Express Jet",
          Carrier == "F9" ~ "Frontier Airlines",
          Carrier == "WN" ~ "Southwest Airlines",
          Carrier == "OO" ~ "SkyWest Airlines"
      )
  ) %>%
    select(Carrier, carrier_name) %>%
    group_by(Carrier, carrier_name) %>%
    tally()

Second, `case_when()` includes an option that cuts down on our work. In particular, say I want to create a new column and label its values as "Weekend" if the DayOfWeek is Saturday or Sunday and "Weekday" if DayOfWeek is any other day. In doing this, it would serve us well to remember that the week begins on Sunday so DayOfWeek == 1 is Sunday, not Monday. 

In [ ]:
cmhflights %>%
  mutate(
      weekend = case_when(
          DayOfWeek %in% c(7, 1) ~ "Yes",
          TRUE ~ "No"
      )
    ) %>%
  select(DayOfWeek, weekend) %>%
  distinct()

Notice how `TRUE` swept up all other values of `DayOfWeek` and coded them as "No."  

One final showcasing of `case_when()`. In **Module 01** we looked at the `hsb2` data and created some `factors` for columns such as female, ses, schtyp, and so on. Well, let us see how the same thing could be done with `case_when()`.

In [ ]:
read.table(
  'https://stats.idre.ucla.edu/stat/data/hsb2.csv',
  header = TRUE,
  sep = ","
  ) -> hsb2

In [ ]:
hsb2 %>%
  mutate(
    female.f = case_when(
      female == 0 ~ "Male",
      female == 1 ~ "Female"),
    race.f = case_when(
      race == 1 ~ "Hispanic",
      race == 2 ~ "Asian",
      race == 3 ~ "African-American",
      TRUE ~ "White"),
    ses.f = case_when(
      ses == 1 ~ "Low",
      ses == 2 ~ "Medium",
      TRUE ~ "High"),
    schtyp.f = case_when(
      schtyp == 1 ~ "Public",
      TRUE ~ "Private"),
    prog.f = case_when(
      prog == 1 ~ "General",
      prog == 2 ~ "Academic",
      TRUE ~ "Vocational")
    ) -> hsb2

hsb2

-----------

## Some other `dplyr()` commands
We have seen `count()` in action but let us see it again, in a slightly different light. In particular, say I want to know how many unique destinations are there connected by air from Columbus. 

### count()

In [ ]:
cmhflights %>%
  filter(Origin == "CMH") %>%
  count(Dest, sort = TRUE)

Note: There is no need for `group_by()` here. And `sort = TRUE` arranges the result in descending order of the frequency (`n`). Here is another code example, this time adding Carrier to the mix. 

In [ ]:
cmhflights %>%
  filter(Origin == "CMH") %>%
  count(Carrier, Dest, sort = TRUE)

How does this help us? Well, now we know that if we were flying to Atlanta, Delta would have the most flights, but if we were flying to the Chicago area then Southwest should be our pick.  

### n_distinct() 
Another useful command is `n_distinct()`, useful in the sense of allowing us to calculate the the number of distinct values of any column. For example, say I want to know how many unique aircraft (not airlines) are there in this data-set. 

In [ ]:
cmhflights %>%
  summarise(n_distinct(TailNum))

### top_n()
If you want to see the top 'n' number of observations, for example the 4 airlines with the most aircraft, you can lean on `top_n()`, as shown below.

In [ ]:
cmhflights %>%
  group_by(Carrier) %>%
  summarise(num.flights = n_distinct(TailNum)) %>%
  arrange(-num.flights) %>% 
  top_n(4)

I am also curious about which aircraft has flown the most, and then maybe 9 other aircraft that follow in descending order.

In [ ]:
cmhflights %>%
  filter(!is.na(TailNum)) %>% # Removing some missing cases 
  group_by(TailNum) %>%
  tally() %>% 
  arrange(-n) %>%
  top_n(4)

### join()
You will, from time to time, need to merge multiple data-sets together. For example, say I have the following data-sets I have created for demonstration purposes. 

In [ ]:
tibble(
  Name = c("Tim", "Tammy", "Bubbles", "Panda"),
  Score = c(5, 8, 9, 10)
    ) -> df1

In [ ]:
tibble(
  Name = c("Tim", "Tammy", "Bubbles"),
  Age = c(25, 78, 19)
    ) -> df2

In [ ]:
tibble(
  Name = c("Tim", "Tammy", "Panda"),
  Education = c("BA", "PhD", "JD")
    ) -> df3

In [ ]:
df1; df2; df3

Notice that Panda is absent from `df2` and Bubbles is absent from `df3`. So if we wanted to build ONE data-set with all data for Tim, Tammy, Bubbles, and Panda, some of the information would be missing for some of these folks. But how could we construct ONE data-set? Via one of a few `join()` commands. 

#### full_join() 
Let us start with a simple full_join, where we link up every individual in df1 or df2 or df3 **regardless of whether they are seen in both data-sets**. 

In [ ]:
df1 %>%
  full_join(df2, by = "Name") %>%
  full_join(df3, by = "Name") 

Pay attention to two things: (i) Name connects the records in each data-set, and so it must be spelled exactly the same for a link to be made, and (ii) the `full_join()` links up all individuals regardless of whether they are missing any information in any of the data-sets. This is usually how most folks will link up multiple files unless they only want records found in a master file. For example, say I want to link up df2 and df3 but only such that the final result will include all records found in BOTH df2 and df3, with df2 serving as the master data-set. Eh?

In [ ]:
df2 %>%
  left_join(df3, by = "Name")  

Notice that Panda is dropped because it is not found in df2. 

Maybe you want df3 to be the master file, in which case you would see a different result (with Bubbles not seen in the result since Bubbles is found in df2 but not in df3): 

In [ ]:
df3 %>%
  left_join(df2, by = "Name")  

Rarely, but definitely not "never," you may want to see the records that are not found in both. Here, anti_join() comes in handy, thus:

In [ ]:
df2 %>%
  anti_join(df3, by = "Name")

In [ ]:
df3 %>%
  anti_join(df2, by = "Name")

--------------------

## Two other useful commands 
### {santoku}
Every now and then you may want to or need to create a grouped version of some numeric variable. For example, we have DepDelay for all flights but want to group this into `quartiles`. How can we do that? In many ways but the easiest might be to use a specific library -- `{santoku}`. Say, for example, I want to create 4 groups of `dep_delay`, and I want these such that we are grouping `DepDelay` into the bottom 25%, next 25%, the next 25%, and finally the highest 25%. Wait, these are the `quartiles`! Fair enough, but how can I do this? 

In [ ]:
library(santoku)

cmhflights %>%
  mutate(
    depdelay_groups = chop_equally(DepDelay, groups = 4)
      ) %>%
  group_by(depdelay_groups) %>%
  tally()

What if we wanted to slice up DepDelay in specific intervals, first at 0, then at 15, then at 30, and then at 45? 

In [ ]:
cmhflights %>%
  mutate(
    depdelay_groups = chop(DepDelay, breaks = c(0, 15, 30, 45))
      ) %>%
  group_by(depdelay_groups) %>%
  tally()

We could also create quintiles (5 groups) or deciles (10 groups) as shown below: 

In [ ]:
cmhflights %>%
  filter(!is.na(DepDelay)) %>%
  mutate(
    depdelay_groups = chop_quantiles(
      DepDelay, c(0.2, 0.4, 0.6, 0.8)
    )
  ) %>%
  group_by(depdelay_groups) %>%
  tally()

In [ ]:
cmhflights %>%
  filter(!is.na(DepDelay)) %>%
  mutate(
    depdelay_groups = chop_quantiles(
      DepDelay, seq(0.1, 0.9, by = 0.1)
    )
  ) %>%
  group_by(depdelay_groups) %>%
  tally()

You could also ask for groups such that they have the same width. Below we create 4 groups. Note that the width of each group is exactly 337.5 ... 
1323 - 985.5 = 337.5; 985.5 - 648 = 337.5; and so on. In this particular example this chopping isn't very useful since we end up with almost all of the data in the very first group. However, there are occasions where equal widths are useful. 

In [ ]:
cmhflights %>%
  filter(!is.na(DepDelay)) %>%
  mutate(
    depdelay_groups = chop_evenly(
        DepDelay,
        4
    )
  ) %>%
  group_by(depdelay_groups) %>%
  tally()

### ordered()
More often than we would like to see happen, we often end up with categorical variables that should follow a certain order but do not. For example, say you have survey data where people were asked to respond whether they Agree, are Neutral, or Disagree with some statement. Let us also assume that the frequencies are as follows:

In [ ]:
tibble(
  response = c(
      rep("Agree", 25), 
      rep("Neutral", 30), 
      rep("Disagree", 45)
      )
    ) -> mydf

In [ ]:
mydf %>%
  group_by(response) %>%
  tally()

Notice how the responses are out of order, with Agree followed by Disagree, then Neutral, since R defaults to alphabetic ordering for anything that is a categorical variable. One way to ensure the correct ordering of categorical variables is via `ordered`, as shown below. 

In [ ]:
mydf %>%
  mutate(
      ordered.response = ordered(
          response,
          levels = c("Agree", "Neutral", "Disagree")
      )
    ) %>%
  group_by(ordered.response) %>%
  tally()

## Concluding thoughts 
We have a covered a lot of ground here but every inch has been critical space. Google any question we have tackled and you will see how many R-users ask the same questions ... how do I calculate mean for groups in R? What you have seen is the heart of the `dplyr()` package. We saw grouped operations, we saw the use of summarise, mutate, case_when, distinct, filter, arrange, select, count, and tally. I will let you in on a secret; while these are core functions, there are others you could experiment with. Look up the cheat-sheet [here](https://rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf). 

Practice what we have done, with the `{nycflights13}` data-set perhaps to get something familiar yet sufficiently different to test your fundamentals. Maybe pick a non-travel data-set altogether, perhaps one of the `tidytuesday` data-sets. What is that you ask? Discover it for yourself [here](https://github.com/rfordatascience/tidytuesday). Bon voyage! Don't go too far because we will be working with two new packages next week -- `{tidyr}` and `{lubridate}`. 

-----

# Exercises for Practice 
## Exercise 01
Why are our best and most experienced employees leaving prematurely? 

The data [available here](https://aniruhil.github.io/avsr/teaching/dataviz/HR_comma_sep.csv) includes information on several current and former employees of an anonymous organization. Fields in the data-set include: 

| Variable | Description |
| :---- | :---- |
| satisfaction_level | = Level of satisfaction (numeric; 0-1) |
| last_evaluation | = Evaluation score of the employee (numeric; 0-1) |
| number_project | = Number of projects completed while at work (numeric) |
| average_monthly_hours | = Average monthly hours spent at the workplace (numeric)  |
| time_spend_company | = Number of years spent in the company (numeric) |
| Work_accident | = Whether the employee had a workplace accident (categorical; 1 = yes or 0 = no) |
| left | = Whether the employee left the workplace or not (categorical; 1 = left or 0 = stayed)  |
| promotion_last_5years | = Whether the employee was promoted in the last five years (categorical; 1 = yes or 0 = no) |
| sales | = Department in which they work (categorical) |
| salary | = Relative level of salary (categorical; low, med, and high) |

(a) Read in the csv-format data-set, naming it `hrdata` and save it in RData format as `hrdata.RData` 

(b) Create new variables that add labels to Work_accident, left, promotion_last_5years, and add these three to `hrdata`.  

(c) Now retain only employees who left the company, and had not been promoted in the last five years. Save this result as `hr01`

(d) In this `hr01` data-set, how many employees do you have per sales department? What sales department has the most number of employees? 

(e) By sales department, calculate mean and standard deviation of (i) satisfaction_level, and (ii) last_evaluation. 

(f) What department has the lowest mean satisfaction? Which department has the highest variation in satisfaction?  

(g) Create a new variable that groups `average_montly_hours` into 4 groups. You can let the group cut-points be chosen automatically with `chop_evenly()`. Then show the frequencies of each group.

## Exercise 02
Thanks to the frenetic work of many individuals, the global spread of the Novel Coronavirus (COVID-19) has been tracked and the data made available for analysis. [Yanchang Zhao](https://rdatamining.wordpress.com/2020/03/10/coronavirus-data-analysis-with-r-tidyverse-and-ggplot2/) is one such individual and for this exercise we will use a spcific version of his data that I have named `cvdata.RData` is available in the `data` folder. Read it in via the `load()` command. We can then answer a few questions. Note the contents: 

+ `country =` name of the country 
+ `date =` date of incidents as recorded 
+ `confirmed =` cumulative count of the number of people who tested positive  
+ `deaths =` cumulative count of the number of people lost to Covid-19 
+ `deaths =` cumulative count of the number of people recovered  

(a) Filter the data-set so that we have only one row per country, the data from March 10, 2020 and call it `cv0310`. 

(b) How many countries have lost `at least one` person to this tragedy? "Others" should not show up as one of the countries.  

(c) What 10 countries have had the most number of confirmed cases? "Others" should not show up as one of the countries. Also ensure the result is organized in descending order of the number of confirmed cases. 

(d) Calculate the `fatality_rate`, defined for our purposes as the percent of deaths. excluding "Others", and only keeping countries that have had `at least 10` confirmed cases, arrange the result to show the top-10 countries in descending order of `fatality_rate`.  

(e) Say we only want to focus on the Baltic countries (Estonia, Latvia, and Lithuania) as a unified group and compare this group to the ASEAN nations (Brunei, Cambodia, Indonesia, Laos, Malaysia, Myanmar, Philippines, Singapore, Thailand, and Vietnam). Use `cv0310` to complete the followng tasks: 

(i) Create a new variable called `region` that only takes on two values -- "Baltic" if the country is a Baltic country and "Asean" if the country is an ASEAN country. 

(ii) Use this variable to calculate the total number of confirmed cases in each region. 